In [2]:
#!wget https://www.electionstudies.org/wp-content/uploads/2017/02/anes_timeseries_cdf_codebook_var.txt

In [287]:
import modpipe
import re
import os

In [288]:
VERSION_RE = re.compile("^RELEASE VERSION:\s+(\d+)")


LINE_SEP = "=" * 77 + "\n"

def defn_iterator(file_path):
    version = None
    lines = []
    
    with open(file_path, errors='ignore') as fp:
        
        for line in fp:
            if version is None:
                version = VERSION_RE.search(line).group(1)
                yield {'version': version}
            elif line == LINE_SEP:
                break
        
        for line in fp:
            if line == LINE_SEP:
                yield lines
                lines = []
            else:
                lines.append(line.rstrip())
                
    if lines[-1] == "" and lines[-2] == "" and lines[-3] == "1":
        yield lines[:-3]

In [289]:
WEIGHTS = set()

DATA_PATH = os.path.join("data", "raw", "anes_timeseries_cdf_codebook_var.txt")

with modpipe.ModPipe("codebook_pipeline") as pipe:
    for lines in defn_iterator(DATA_PATH):
        if 'version' not in lines:
            res = pipe(lines.copy())
            if '_general_note_lines' not in res:
                if 'sections' not in res:
                    continue
                for weight in res['sections'].get('WEIGHT', []):
                    WEIGHTS.add(weight)
                pass
                #TYPES.update(res['sections']['TYPE'])
                #if len(res['sections']['QUESTION']) == 10:
                #    break
                #pass

TYPES

set()

In [290]:
res

OrderedDict([('var_name', 'VCF9155'),
             ('desc_kind', 'VOTE VALIDATION'),
             ('desc', 'Was R Vote Validated'),
             ('var_type', 'Numeric  Dec 0-1'),
             ('question', [' ']),
             ('valid_codes',
              OrderedDict([('UNIFORM',
                            OrderedDict([('1', ' '),
                                         ('3', ' '),
                                         ('5', ' ')]))])),
             ('missing_codes',
              OrderedDict([('UNIFORM',
                            OrderedDict([('0', ' '),
                                         ('9', ' '),
                                         ('INAP', ' ')]))])),
             ('source_vars',
              OrderedDict([('1964', ['V640423', 'V640424', 'V64042']),
                           ('1976', ['V765002', 'V76501']),
                           ('1978', ['V781400', 'V781409', 'V78141']),
                           ('1980', ['V80120']),
                           ('1984', 

In [259]:
import copy


SOURCE_VAR_RE = re.compile("^(\d{4}): (.*)$")


def parse_source_vars(sections, var_def):
    lines = sections.pop('SOURCE_VARS')
    if lines is None:
        return
    
    source_vars = OrderedDict()
    for line in lines:
        m = SOURCE_VAR_RE.match(line)
        assert m, line
        source_vars[m.group(1)] = [s.strip() for s in m.group(2).split(",")]
        
    var_def['source_vars'] = source_vars

        
parse_source_vars(copy.deepcopy(res['sections']), {})

In [249]:
res['sections']

OrderedDict([('NOTES',
              ['See note VCF9149,VCF9152.',
               'GENERAL NOTE:',
               'Code 3 and code 5 include cases of limited access to vote records',
               '(codes 2 and 5 in VCF9153).  Codes 3 and 5 also incorporate 1984 and',
               '1990 cases of attempted validation by phone of Rs registered out of',
               'area (998 in VCF9149).',
               'In cases where registration is not required or where voting and',
               'registration lists are for some reason kept entirely separate, this',
               'code implies that an independent check for voting was also done and no',
               'record of R voting was found.',
               'In all Pre-Post years except 1964 (and some cases in 1976), this',
               'variable includes in valid codes respondents who did not give a post-',
               'election interview.',
               '1964 and 1980 are the only years with cases categorized for same-day',
  